In [5]:
# 7. Optimization via Gene Expression Algorithms:
# Application - Smart grids and energy optimization

import random
import numpy as np

class SmartGridOptimization:
    def __init__(self):
        # User input
        self.demand = float(input("Enter total energy demand (in kWh): "))
        self.solar_capacity = float(input("Enter solar energy capacity (in kWh): "))
        self.conventional_capacity = float(input("Enter conventional power plant capacity (in kWh): "))
        self.solar_cost_per_kWh = float(input("Enter cost per kWh of solar energy ($): "))
        self.conventional_cost_per_kWh = float(input("Enter cost per kWh of conventional energy ($): "))

        # Algorithm parameters
        self.population_size = 20
        self.num_generations = 50
        self.mutation_rate = 0.1
        self.crossover_rate = 0.9
        self.num_genes = 2  # Solar and conventional energy
        self.best_solution = None

    def generate_initial_population(self):
        # Generate initial population using NumPy for efficiency
        solar_energy = np.random.uniform(0, self.solar_capacity, self.population_size)
        conventional_energy = np.random.uniform(0, self.conventional_capacity, self.population_size)
        population = np.column_stack((solar_energy, conventional_energy))
        return population

    def fitness_function(self, population):
        # Compute fitness for the entire population using vectorized operations
        solar_energy = population[:, 0]
        conventional_energy = population[:, 1]
        total_energy = solar_energy + conventional_energy

        # Apply penalty for unmet demand
        penalty = np.maximum(0, self.demand - total_energy) * 1000

        # Compute total cost
        solar_cost = solar_energy * self.solar_cost_per_kWh
        conventional_cost = conventional_energy * self.conventional_cost_per_kWh
        total_cost = solar_cost + conventional_cost + penalty

        return total_cost

    def select_parents(self, population, fitness):
        # Probabilistic selection using roulette wheel
        probabilities = 1 / (fitness + 1e-9)  # Avoid division by zero
        probabilities /= probabilities.sum()
        selected_indices = np.random.choice(len(population), size=2, p=probabilities, replace=False)
        return population[selected_indices]

    def crossover(self, parent1, parent2):
        # Single-point crossover
        if random.random() < self.crossover_rate:
            point = random.randint(1, self.num_genes - 1)
            child1 = np.hstack((parent1[:point], parent2[point:]))
            child2 = np.hstack((parent2[:point], parent1[point:]))
            return child1, child2
        return parent1, parent2

    def mutate(self, individual):
        # Random mutation within capacity limits
        if random.random() < self.mutation_rate:
            gene = random.randint(0, self.num_genes - 1)
            if gene == 0:  # Mutate solar energy
                individual[gene] = random.uniform(0, self.solar_capacity)
            else:  # Mutate conventional energy
                individual[gene] = random.uniform(0, self.conventional_capacity)
        return individual

    def evolve(self):
        # Initialize population
        population = self.generate_initial_population()

        for generation in range(self.num_generations):
            # Compute fitness for the population
            fitness = self.fitness_function(population)

            # Update best solution
            best_idx = np.argmin(fitness)
            if self.best_solution is None or fitness[best_idx] < self.fitness_function(np.array([self.best_solution]))[0]:
                self.best_solution = population[best_idx]

            # Early stopping if fitness converges
            if fitness[best_idx] < 1e-3:  # Converged to near-zero cost
                break

            # Create new generation
            new_population = [self.best_solution]  # Elitism: carry over the best solution

            while len(new_population) < self.population_size:
                parent1, parent2 = self.select_parents(population, fitness)
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutate(child1))
                if len(new_population) < self.population_size:
                    new_population.append(self.mutate(child2))

            # Convert new population to NumPy array for the next generation
            population = np.array(new_population)

        return self.best_solution

# Main execution
if __name__ == "__main__":
    optimizer = SmartGridOptimization()
    best_solution = optimizer.evolve()

    solar_energy, conventional_energy = best_solution
    total_cost = optimizer.fitness_function(np.array([best_solution]))[0]

    print("\nBest solution found:")
    print(f"Solar energy generated: {solar_energy:.2f} kWh")
    print(f"Conventional energy generated: {conventional_energy:.2f} kWh")
    print(f"Total energy generated: {solar_energy + conventional_energy:.2f} kWh")
    print(f"Total cost: ${total_cost:.2f}")

print("Sparsha Srinath Kadaba - 1BM22CS287")

Enter total energy demand (in kWh): 200
Enter solar energy capacity (in kWh): 100
Enter conventional power plant capacity (in kWh): 100
Enter cost per kWh of solar energy ($): 50
Enter cost per kWh of conventional energy ($): 30

Best solution found:
Solar energy generated: 99.00 kWh
Conventional energy generated: 96.87 kWh
Total energy generated: 195.87 kWh
Total cost: $11983.45
Sparsha Srinath Kadaba - 1BM22CS287
